# Pathfinding in Boston

A partir de l'algorithme $A^*$ vu en cours, et des fichiers boston/boston_ways.geojson et boston/boston_vertices.geojson :
- Choisir aléatoirement 2 points,
- Calculer le chemin le plus court entre ces 2 points,
- Afficher le résultat.

In [1]:
import folium
import geopandas as gpd
import random

In [2]:
class Node():
    """Classe Node.
    
    Arguments :
    vertice_id : identifiant du noeud
    cost : distance réellement parcouru depuis le départ (0 par défaut)
    heuristic : estimation de la distance à parcourir jusqu'à destination (0 par défaut)
    parent : référence vers le noeud parent dans l'arbre de recherche (None par défaut)
    """
    def __init__(self, vertice_id, cost = 0, heuristic = 0, parent = None):
        self.vertice_id = vertice_id
        self.cost = cost
        self.heuristic = heuristic
        self.parent = parent
     
    # 2 noeuds sont égaux s'ils ont le même id
    def __eq__(self, n):
        return self.vertice_id == n.vertice_id
    
    def __str__(self):
        return str(self.vertice_id)
    
    # Construit la liste des noeuds jusqu'à la racine
    def get_path(self):
        if self.parent is None:
            return [self.vertice_id]
        else:
            return self.parent.get_path() + [self.vertice_id]

In [3]:
def cost(u_id, v_id, ways):
    """Retourne la longueur de l'arc (u_id, v_id) dans ways."""
    
    # Cas où dans ways source == u_id et target == v_id
    c = ways[(ways['source'] == u_id) & (ways['target'] == v_id)]
    if not c.empty:
        return c.iloc[0]['length_m']
    # Sinon l'arc existe dans le sens opposé
    else:
        c = ways[(ways['target'] == u_id) & (ways['source'] == v_id)].iloc[0]['length_m']
        return c

In [4]:
def distance(u, v):
    """Distance catésienne en mètres entre les points u et v dans le plan."""
    return u.distance(v)

In [5]:
def children(point_id, ways):
    """Distance liste des noeuds accessibles depuis point_id."""
    return ways[ways['source'] == point_id]['target'].to_list() + ways[ways['target'] == point_id]['source'].to_list()

In [6]:
def astart(depart_id, destination_id, ways, vertices):
    """A* calcule le plus court chemin entre depart_id et destination_id."""
    
    depart = Node(depart_id)
    destination = Node(destination_id)
    point_destination = vertices[vertices['id'] == destination_id].iloc[0]['geometry']
    
    closeList = []
    openList = [depart]
    
    while openList:
        best = openList.pop(0)
        closeList.append(best)
        if best == destination:
            return best.get_path()
        for voisin in children(best.vertice_id, ways):
            voisin_cost = best.cost + cost(best.vertice_id, voisin, ways)
            point_voisin = vertices[vertices['id'] == voisin].iloc[0]['geometry']
            voisin_heuristic = distance(point_voisin, point_destination)
            node_voisin = Node(voisin, cost = voisin_cost, heuristic = voisin_heuristic, parent = best)
            if not (node_voisin in closeList):
                openList.append(node_voisin)
                # Trie les noeuds de openList selon leurs valeurs cost + heuristic
                openList.sort(key = lambda x : x.cost + x.heuristic)
    return []

In [7]:
# Graphe/plan de circulation de la ville de Boston
ways = gpd.read_file("boston/boston_ways.geojson")
# Noeuds du graphe de Boston
vertices = gpd.read_file("boston/boston_vertices.geojson")
# Projection de EPSG:4326 vers EPSG:26986 pour calculs de distances cartésiennes
ways = ways.to_crs('EPSG:26986')
vertices = vertices.to_crs('EPSG:26986')

In [8]:
# Tirage aléatoire des noeuds de départ et d'arrivée
depart = vertices.loc[random.randint(0, len(vertices) - 1)]['id']
destination = vertices.loc[random.randint(0, len(vertices) - 1)]['id']
# Calcule du plus court chemin avec A*
shortest_path = astart(depart, destination, ways, vertices)

In [9]:
# Carte Folium centrée sur Boston
boston = folium.Map(location = [42.361145, -71.057083], zoom_start = 13)

In [10]:
# Liste des arcs du plus court chemin
edges = []

for i in range(0, len(shortest_path) - 1):
    # Extraction de u et de son successeur v
    u = shortest_path[i]
    v = shortest_path[i + 1]
    # Extraction de l'arc (u, v)
    edge = ways[(ways['source'] == u) & (ways['target'] == v)]
    if not edge.empty:
        e = edge.iloc[0]['geometry']
    else:
        e = ways[(ways['source'] == v) & (ways['target'] == u)].iloc[0]['geometry']
    edges.append(e)

# Construction d'un geodataframe à partir des arcs avec EPSG:26986
df = gpd.GeoDataFrame({"geometry" : edges}).set_crs('EPSG:26986')
# Reprojection en EPSG:4326 pour affichage sur carte Folium
p = df.to_crs('EPSG:4326')
# Conversion au format geojson
edges_json = p.to_json()

In [11]:
folium.GeoJson(data = edges_json, name = "Path").add_to(boston)
folium.LayerControl().add_to(boston)

In [12]:
boston